In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from tkinter import Tk, filedialog
import sys
import pandas as pd
from matplotlib.animation import FuncAnimation

# --- Configuración inicial ---
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12

# --- Función para seleccionar video ---
print("Seleccione el video:")
def select_video():
    root = Tk()
    root.withdraw()
    video_path = filedialog.askopenfilename(
        title="Selecciona el video ecocardiográfico",
        filetypes=[("Videos", "*.avi *.mp4 *.mov"), ("Todos los archivos", "*.*")]
    )
    root.destroy()
    return video_path

# --- Cargar video ---
video_path = select_video()
if not video_path:
    print("No se seleccionó ningún archivo.")
    sys.exit()

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error al abrir el video. Verifica:")
    print(f"- Ruta: {video_path}")
    print("- Formato (convierte a MP4 si es necesario)")
    print("- Instala codecs con: pip install ffmpeg-python")
    sys.exit()

fps = cap.get(cv2.CAP_PROP_FPS)
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"\nVideo cargado: {video_path}")
print(f"Resolución: {int(cap.get(3))}x{int(cap.get(4))}")
print(f"FPS: {fps:.2f}, Total frames: {num_frames}")

# --- Selección manual de puntos ---
ret, frame1 = cap.read()
if not ret:
    print("Error al leer el primer frame.")
    sys.exit()

frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Configurar matplotlib para modo interactivo
plt.switch_backend('TkAgg')
plt.imshow(frame1_gray, cmap='gray')
plt.title("Seleccione 2 puntos:\n1. Pared septal (Click 1)\n2. Pared lateral (Click 2)")
print("\nPor favor, seleccione 2 puntos en la imagen...")
points = plt.ginput(2, timeout=0)
plt.close()

if len(points) < 2:
    print("Error: Debes seleccionar exactamente 2 puntos.")
    sys.exit()

x1, y1 = int(points[0][0]), int(points[0][1])
x2, y2 = int(points[1][0]), int(points[1][1])
roi_size = 10

print(f"\nPuntos seleccionados:")
print(f"- Pared septal: ({x1}, {y1})")
print(f"- Pared lateral: ({x2}, {y2})")

# --- Extraer parches de referencia ---
patch1 = frame1_gray[y1-roi_size:y1+roi_size, x1-roi_size:x1+roi_size]
patch2 = frame1_gray[y2-roi_size:y2+roi_size, x2-roi_size:x2+roi_size]

if patch1.size == 0 or patch2.size == 0:
    print("Error: Los puntos seleccionados están demasiado cerca del borde.")
    sys.exit()

# --- Variables para almacenamiento y visualización ---
displacements1, displacements2 = [y1], [y2]
frames = []
tracking_results = []

# --- Función para procesar cada frame ---
def process_frame(i):
    global cap, displacements1, displacements2
    
    if i == 0:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 1)  # Empezar desde el frame 1
    
    ret, frame = cap.read()
    if not ret:
        return None
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Seguimiento por correlación cruzada
    c1 = cv2.matchTemplate(gray_frame, patch1, cv2.TM_CCOEFF_NORMED)
    c2 = cv2.matchTemplate(gray_frame, patch2, cv2.TM_CCOEFF_NORMED)
    
    _, max_val1, _, max_loc1 = cv2.minMaxLoc(c1)
    _, max_val2, _, max_loc2 = cv2.minMaxLoc(c2)
    
    if max_val1 > 0.5 and max_val2 > 0.5:
        y1_track = max_loc1[1] + roi_size
        y2_track = max_loc2[1] + roi_size
    else:
        y1_track = displacements1[-1]
        y2_track = displacements2[-1]
    
    displacements1.append(y1_track)
    displacements2.append(y2_track)
    
    # Dibujar resultados en el frame (con cruces en lugar de círculos)
    frame_display = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2BGR)
    
    # Rectángulos de las ROI
    cv2.rectangle(frame_display, (x1-roi_size, y1-roi_size), 
                 (x1+roi_size, y1+roi_size), (255, 0, 0), 1)
    cv2.rectangle(frame_display, (x2-roi_size, y2-roi_size), 
                 (x2+roi_size, y2+roi_size), (0, 255, 0), 1)
    
    # Dibujar cruces para el seguimiento
    cross_size = 3
    # Cruz para punto 1 (rojo)
    cv2.line(frame_display, (x1-cross_size, int(y1_track)), 
            (x1+cross_size, int(y1_track)), (0, 0, 255), 1)
    cv2.line(frame_display, (x1, int(y1_track)-cross_size), 
            (x1, int(y1_track)+cross_size), (0, 0, 255), 1)
    
    # Cruz para punto 2 (verde)
    cv2.line(frame_display, (x2-cross_size, int(y2_track)), 
            (x2+cross_size, int(y2_track)), (0, 255, 0), 1)
    cv2.line(frame_display, (x2, int(y2_track)-cross_size), 
            (x2, int(y2_track)+cross_size), (0, 255, 0), 1)
    
    # Mostrar frame con seguimiento
    cv2.imshow('Seguimiento en Tiempo Real', frame_display)
    if cv2.waitKey(int(1000/fps)) & 0xFF == ord('q'):
        return None
    
    return frame_display

# --- Procesamiento del video con visualización ---
print("\nIniciando procesamiento con visualización en tiempo real...")
print("Presione 'q' para salir de la visualización")

# Configurar ventana de visualización
cv2.namedWindow('Seguimiento en Tiempo Real', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Seguimiento en Tiempo Real', 800, 600)

try:
    for i in range(1, num_frames):
        result = process_frame(i)
        if result is None:
            break
except KeyboardInterrupt:
    print("\nProcesamiento interrumpido por el usuario")
finally:
    cap.release()
    cv2.destroyAllWindows()

# --- Cálculo de deformación y strain rate ---
displacements1 = np.array(displacements1)
displacements2 = np.array(displacements2)

defor1 = (displacements1 - displacements1[0]) / displacements1[0]
defor2 = (displacements2 - displacements2[0]) / displacements2[0]

dt = 1 / fps
strain_rate1 = np.diff(defor1) / dt
strain_rate2 = np.diff(defor2) / dt

# --- Resultados ---
print("\n--- Resultados ---")
print(f"Deformación máxima (Septal): {np.min(defor1):.4f}")
print(f"Deformación máxima (Lateral): {np.min(defor2):.4f}")
print(f"Strain Rate máximo (Septal): {np.min(strain_rate1):.2f} 1/s")
print(f"Strain Rate máximo (Lateral): {np.min(strain_rate2):.2f} 1/s")

# --- Visualización Completa ---
plt.figure(figsize=(15, 10))

# 1. Frame inicial con puntos seleccionados
plt.subplot(2, 2, 1)
plt.imshow(frame1_gray, cmap='gray')
plt.scatter(x1, y1, c='r', s=50, label='Septal')
plt.scatter(x2, y2, c='b', s=50, label='Lateral')
plt.title("Puntos iniciales seleccionados")
plt.legend()

# 2. Gráfico de deformación (Strain)
plt.subplot(2, 2, 2)
plt.plot(defor1, 'r', label='Septal', linewidth=2)
plt.plot(defor2, 'b', label='Lateral', linewidth=2)
plt.title("Deformación Normalizada (Strain)")
plt.xlabel("Frame")
plt.ylabel("Strain")
plt.legend()
plt.grid()

# 3. Gráfico de tasa de deformación (Strain Rate)
plt.subplot(2, 2, 3)
time = np.arange(len(strain_rate1)) * dt
plt.plot(time, strain_rate1, 'r', label='Septal', linewidth=2)
plt.plot(time, strain_rate2, 'b', label='Lateral', linewidth=2)
plt.title("Tasa de Deformación (Strain Rate)")
plt.xlabel("Tiempo (s)")
plt.ylabel("1/s")
plt.legend()
plt.grid()

# 4. Frame final con puntos de seguimiento
cap_final = cv2.VideoCapture(video_path)
final_frame_idx = min(len(displacements1)-1, num_frames-1)
cap_final.set(cv2.CAP_PROP_POS_FRAMES, final_frame_idx)
ret, final_frame = cap_final.read()

plt.subplot(2, 2, 4)
if ret and final_frame is not None:
    final_gray = cv2.cvtColor(final_frame, cv2.COLOR_BGR2GRAY)
    plt.imshow(final_gray, cmap='gray')
    plt.scatter(x1, displacements1[-1], c='r', s=50, label='Septal final')
    plt.scatter(x2, displacements2[-1], c='b', s=50, label='Lateral final')
else:
    plt.text(0.5, 0.5, "Frame final no disponible", ha='center', va='center')
plt.title("Posiciones finales de seguimiento")
plt.legend()

plt.tight_layout()

try:
    plt.show()
except KeyboardInterrupt:
    print("\nVisualización de gráficos interrumpida")

# Liberar recursos
cap_final.release()
cv2.destroyAllWindows()

Seleccione el video:

Video cargado: C:/Users/geron/OneDrive/Escritorio/documentos/Eko/0X1A296F5FCD5A0ED8.avi
Resolución: 112x112
FPS: 50.00, Total frames: 131

Por favor, seleccione 2 puntos en la imagen...

Puntos seleccionados:
- Pared septal: (55, 76)
- Pared lateral: (91, 68)

Iniciando procesamiento con visualización en tiempo real...
Presione 'q' para salir de la visualización

--- Resultados ---
Deformación máxima (Septal): -0.3816
Deformación máxima (Lateral): -0.8529
Strain Rate máximo (Septal): -27.63 1/s
Strain Rate máximo (Lateral): -58.82 1/s
